In [1]:
import numpy as np
import cancer
import sklearn.preprocessing

In [3]:
X_train, y_train = cancer.get_train()
X_train = sklearn.preprocessing.scale(X_train)
X_test, y_test = cancer.get_test()
X_test = sklearn.preprocessing.scale(X_test)

In [6]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

clf = NearestCentroid()
clf.fit(X_train, y_train)
np.mean(clf.predict(X_test) == y_test)

0.37037037037037035

In [4]:
def fit(X, y):
    N, D = X.shape
    cs = np.unique(y)
    C = len(cs)
    sse = np.zeros(D)
    mus = []
    prior = []

    for c in cs:
        i = y == c
        X_c = X[i]
        prior.append(np.mean(i))
        mu_c = np.mean(X_c, axis = 0)
        mus.append(mu_c)
        sse += np.sum((X_c - mu_c) ** 2, axis = 0)

    sigma2 = sse / (1.0 * N - C)
        
    return cs, np.log(prior), mus, sigma2
        
def predict(model, X):
    cs, log_prior, mus, sigma2 = model
    N, D = X.shape
    C = len(cs)
    loglik = np.empty((N, C))
    
    for i in range(C):
        Z = 0.5 * (X - mus[i]) ** 2 / sigma2
        loglik[:, i] = log_prior[i] - np.sum([Z[:, j] for j in range(D)], axis = 0)
    
    return cs[np.argmax(loglik, axis = 1)]    

In [5]:
model = fit(X_train, y_train)
np.mean(predict(model, X_test) == y_test)

0.37037037037037035